### Process Races

In [1]:
from formula1.formula1_constants import *
from formula1.formula1_utils import *

##### Step 1 - Read from Bronze Layer

In [2]:
races_df = spark.read.table("f1_bronze.races")

##### Step 2 - Transform and Select Columns
- Rename ID columns
- Create race_timestamp

In [3]:
from pyspark.sql.functions import col, to_timestamp, concat, lit

# The Bronze races table might already have columns from the CSV read
# raceId, year, round, circuitId, name, date, time, url
# It also has Bronze metadata (ingestion_date, etc.), but we might want to re-ingest or keep it.
# Usually Silver just adds its own processing time.

races_transformed_df = races_df.withColumn(
    "race_timestamp",
    to_timestamp(concat(col("date"), lit(" "), col("time")), "yyyy-MM-dd HH:mm:ss"),
)

races_selected_df = races_transformed_df.select(
    col("raceId").alias("race_id"),
    col("year").alias("race_year"),
    col("round"),
    col("circuitId").alias("circuit_id"),
    col("name"),
    col("race_timestamp"),
)

##### Step 3 - Add Ingestion Date

In [4]:
races_final_df = add_ingestion_date(races_selected_df)

##### Step 4 - Write to Silver Layer

In [5]:
races_final_df.write.mode("overwrite").partitionBy("race_year").format(
    "delta"
).saveAsTable("f1_silver.races")